In [ ]:
"""
task1: 短期实习3&4
autor: 邵宇辉202083300563 20级气象学(8)班
time: 2023/04/14
"""
import numpy as np
import pandas as pd
from cartopy.util import add_cyclic_point
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.colors as mcolors
from scipy.stats import pearsonr
import cartopy.io.shapereader as shpreader
import xarray as xr
from sklearn.preprocessing import scale
from scipy.interpolate import Rbf#引入径向基函数
import matplotlib.colors as colors
import geopandas as gpd
plt.rcParams['font.sans-serif']=['KaiTi']#英文新罗马字体
plt.rcParams['axes.unicode_minus']=False

In [ ]:
def Corr_vars(NAO_index, DJF_var,lon= False,lat= False):
    # 遍历每个格点, 求出其pearson相关系数
    r_p = np.zeros((2,len(lat.values), len(lon.values)))  #用于存储格点的相关系数&P-value
    for m, n in enumerate(lon.values):
        for i,j in enumerate(lat.values):
            r_p[0][i][m] = pearsonr(DJF_var[:,i, m], NAO_index)[0]     #r
            r_p[1][i][m] = pearsonr(DJF_var[:,i, m], NAO_index)[1]     #p-value
    return r_p

def creat_map(extent):  #底图绘制
    ##绘图区域
    proj = ccrs.PlateCarree(central_longitude= 180)
    fig = plt.figure(figsize=(6,8), dpi= 300, facecolor='white')
    ax = fig.add_axes([0.1,0.2,0.9,0.4], projection= proj)
    ##设置地图属性
    # states_provinces = cfeature.NaturalEarthFeature(
    #                     category='cultural',
    #                     name='admin_1_states_provinces_lines',
    #                     scale='50m',
    #                     facecolor='none')
    # ax.add_feature(states_provinces, edgecolor='k',linewidth= 0.6, zorder= 2)
    ax.add_feature(cfeature.LAND, facecolor='#D3D3D3')
    ax.add_feature(cfeature.OCEAN, facecolor='never')
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth= 0.6, zorder= 2)
    # ax.add_feature(cfeature.BORDERS, linestyle='-',linewidth= 0.6,zorder= 2)
    ax.set_extent(extent)  #范围
    return fig, ax, proj


def truncate_colormap(cmap, minval=0.4, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        "trunc({n},{a:.2f},{b:.2f})".format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)),
    )
    return new_cmap

#### 1.EU遥相关指数

In [ ]:
# 数据读取
data = xr.open_dataset(r'./era5_geo500_monthly_1951_2010_avedata.nc')
EU_Jan = -data.z.loc[:,55,180+20]/9.8/4 + data.z.loc[:,55,180+75]/9.8/2 - data.z.loc[:,40,180+145]/9.8/4     #1951-2010年500hPa1月EU遥相关指数
EU_Jan_scale = scale(EU_Jan)

In [ ]:
# 滑动平均折线图
years = np.arange(1951, 2010+1)
import proplot as pplt
fig, ax = pplt.subplots(
                        refheight = 2,
                        refwidth= 5,
                        dpi= 300,
                        xlabel='Year',
                        ylabel='EU Index(scaled)',
                        share= 1
)
ax[0].plot(years, EU_Jan_scale, color= '#8b2671', marker='o',markersize=3, lw=1.5)
ax.format(
          ylim=(-3,3),
          xlim=(1949, 2012),
          yticks= 1,
          xticks= 10,
          linewidth= 1.2,
          abcsize= 14,
          titlesize= 14,
          xlabelsize= 10,
          ylabelsize= 10
)
fig.savefig(r'../PIC/P3_1951_2010_500hPa_EU_index(JanAve_scale).svg', format='svg', dpi=300, pad_inches= 0.2, bbox_inches= 'tight')

#### 2.EU指数与环流场相关性

In [ ]:
# 数据读取
hgt = np.array(data.z)
r_p = Corr_vars(EU_Jan, hgt, data.longitude, data.latitude)

In [ ]:
# 绘图
fig, ax , proj = creat_map(extent=[-180, 180, -90, 90])   #绘图
# 填色 显著性
C = ax.contourf(data.longitude,  data.latitude, r_p[0], transform= proj,cmap='bwr', extend='neither', zorder=1, levels= np.arange(-1,1.1,0.1))
C1 = ax.contourf(data.longitude,  data.latitude, r_p[1], [0, 0.05, 1], zorder=1, hatches=['\\', None], colors='none',transform= proj)
C2 = ax.contourf(data.longitude,  data.latitude, r_p[1], [0, 0.05, 1], zorder=1, hatches=['//', None], colors='none',transform= proj)
# 标签
ax.set_xticks([-180,-90, 0,90, 180], crs= proj)
ax.set_yticks([-90,-60,-30, 0, 30,60,90], crs= proj)
ax.xaxis.set_major_formatter(LongitudeFormatter())
ax.yaxis.set_major_formatter(LatitudeFormatter())
cbar_ax = fig.add_axes([0.15,0.172,0.8,0.022])
fig.colorbar(C,
        orientation= 'horizontal', cax= cbar_ax,
        ticks= np.arange(-1,1.2,0.2)
        )
fig.savefig(r'../PIC/P3_1951_2010_500hPa_corr(EU&hgt).svg',format='svg', dpi=300, pad_inches= 0.3, bbox_inches= 'tight' )

#### 3.1月份WP遥相关指数与我国气温的相关系数分布

In [ ]:
# 读取数据
data_tep = pd.read_table(r'./t1601.txt', header= None)
data_sta = pd.read_excel(r'./station160.xlsx')
data_sta1 = pd.DataFrame(np.zeros((160, 5+60)), columns=list(['number', 'name', 'number1', 'LAT', 'LON']) + list( np.arange(1951, 2010+1)))
data_sta1.iloc[:,0:5] = data_sta
# 存储
data_tep = data_tep.iloc[:,0].apply(lambda x: np.array(str(x).split()))
for i in np.arange(2010-1951+1):    #年循环
    data_tep_year = data_tep.iloc[i*8:i*8+8]
    for j in np.arange(8):          #站点循环
        data_sta1.iloc[j*20:j*20+20, i+5] = data_tep_year.iloc[j].astype(int)

#计算与Nino3.4 Index的相关系数
data_sta1['corr_EU&T'] = data_sta1.iloc[:, 5:5+60].apply(lambda x: pearsonr(x, EU_Jan)[0], axis= 1)   #Corr(WHD, Nino34_index)
data_sta1['P_EU&T'] = data_sta1.iloc[:, 5:5+60].apply(lambda x: pearsonr(x, EU_Jan)[1], axis= 1) #P-value

In [ ]:
# 站点->格点
data_corr = data_sta1.iloc[:,[3,4,-2,-1]]
lon = data_corr.LON
lat = data_corr.LAT
corr = data_corr['corr_EU&T']
P = data_corr['P_EU&T']
# 插值
olon = np.linspace(70, 140,100)
olat = np.linspace(15, 55, 100)
olon, olat = np.meshgrid(olon, olat)
func = Rbf(lon, lat, corr, function='linear')
func1 = Rbf(lon, lat, P, function='linear')
corr_new = func(olon, olat)    #插值结果
P_new = func1(olon, olat)
# geopandas方法裁剪中国区域
data_corr_df = pd.DataFrame(dict(lon= olon.flatten(), lat= olat.flatten()))
data_corr_df['corr'] = corr_new.flatten()
data_corr_df['P'] = P_new.flatten()
df_geo = gpd.GeoDataFrame(data_corr_df,
                          geometry= gpd.points_from_xy(data_corr_df['lon'], data_corr_df['lat']),
                          crs= 'EPSG:4326')
china_shp = gpd.read_file(r'../map/china.shp', encoding= 'UTF-8')
df_clip = gpd.clip(df_geo, china_shp)

# 数据再处理
shp_china = shpreader.Reader(r'../map/china.shp')
shp_prov = shpreader.Reader(r'D:\DaChuang\Data\shp\cn_shp\Province_9\Province_9.shp')
df_clip1 = df_clip[df_clip['P']<0.1]

# 绘图
##基础部分
proj = ccrs.PlateCarree()
fig = plt.figure(figsize=(6,8), dpi= 300, facecolor='white')
ax = fig.add_axes([0.1,0.2,0.9,0.4], projection= proj)
ax.add_geometries(shp_china.geometries(),crs=ccrs.PlateCarree(),edgecolor='k',facecolor='none',lw=1)
ax.add_geometries(shp_prov.geometries(),crs=ccrs.PlateCarree(),edgecolor='k',facecolor='none',lw=0.5)
ax.set_extent([70,140,15,55],crs=ccrs.PlateCarree())
ax.set_xticks(np.arange(80,150,20))
ax.set_yticks(np.arange(20,60,10))
ax.xaxis.set_major_formatter(LongitudeFormatter())
ax.yaxis.set_major_formatter(LatitudeFormatter())
##白化部分
ac = ax.scatter(df_clip['lon'],df_clip['lat'], c= df_clip['corr'], s=9,marker='s',
                cmap= truncate_colormap(plt.cm.bwr, minval=0, maxval= 1),
                norm = mcolors.TwoSlopeNorm(vmin=-0.3, vmax = 0.3, vcenter=0) )
ax.scatter(df_clip1['lon'],df_clip1['lat'], c= 'black', s=9,marker='.')
cbar_ax = fig.add_axes([0.15,0.14,0.8,0.022])
fig.colorbar(ac,orientation= 'horizontal', cax= cbar_ax,ticks= np.linspace(-0.3, 0.3,4+1))
fig.savefig(r'../PIC/P3_1951_2010_160sta_corr(EU&temp).svg',format='svg', dpi=300, pad_inches= 0.3, bbox_inches= 'tight' )